In [2]:
import django
import sys, os
from gensim.models import Word2Vec, Doc2Vec
from gensim.models import Phrases
import gensim

sys.path.append('/home/galm/software/django/tmv/BasicBrowser/')
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()

from scoping.models import *

import pandas as pd
import numpy as np

q = Query.objects.get(pk=1457)

from mongoengine import *
connect('mongoengine_documents')
import matplotlib.pyplot as plt
from mongoengine.queryset.visitor import Q
from monary import Monary
mon = Monary()

from scoping.models import *

from mongo_classes import *

In [3]:
docs = Doc.objects.filter(title__iregex='\w').order_by('id')#.filter(query=q)


In [4]:
new_model = True

if new_model:
    def read_docs(fname, tokens_only=False):
        for i, doc in enumerate(docs.iterator()):
            yield gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(doc.title), [doc.pk])

    titles = list(read_docs(docs))   
    model = Doc2Vec(vector_size=150, min_count=4, epochs=20)
    model.build_vocab(titles)
    %time model.train(titles, total_examples=model.corpus_count, epochs=model.epochs)
    model.save("/usr/local/apsis/queries/title_model_150")
else:
    model = Doc2Vec.load("/usr/local/apsis/queries/title_model")

In [64]:
doc = "Community care practices in Papua New Guinea"
#doc = "the community connection  implementation of best evidence into practice for self management of chronic diseases"
inferred_vector = model.infer_vector(
    gensim.utils.simple_preprocess(doc),
    steps=10
)
sims = model.docvecs.most_similar([inferred_vector], topn=10)

for label, index in [('MOST', 0), ('SECOND-MOST', 1), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    doc = Doc.objects.get(pk=sims[index][0])
    print(u'%s %s: «%s»\n' % (label, sims[index], doc.title))


MOST (732190, 0.6620210409164429): «Stress in community care teams: will it affect the sustainability of community care?»

SECOND-MOST (536906, 0.6337926387786865): «Trends in Contraceptive and Preconception Care in United States Ambulatory Practices»

MEDIAN (1409338, 0.5917409062385559): «RECOMMENDATIONS FOR THE CARE OF REPTILES AND AMPHIBIANS»

LEAST (1226785, 0.5852288007736206): «Does the UK gas industry care?»



In [54]:
d = "Community care practices in Papua New Guinea"
dv = model.infer_vector(gensim.utils.simple_preprocess(d),steps=10)
#print(dv)
d2id = 732190
d2 = Doc.objects.get(pk=d2id).title

d2v = model.docvecs[d2id]

print(d2)

dv_sim = model.docvecs.similarity_unseen_docs(
    model,
    gensim.utils.simple_preprocess(d), 
    gensim.utils.simple_preprocess(d2),
    steps=10
)

dv_sim

print(dv_sim)

Stress in community care teams: will it affect the sustainability of community care?
0.47854826


In [52]:
import scipy
scipy.spatial.distance.cosine(dv,d2v)

0.4411402218441035

In [56]:
model.docvecs.closer_than(dv,d2v)

TypeError: unhashable type: 'numpy.ndarray'

In [11]:
doc = Doc.objects.last().title
doc = gensim.utils.simple_preprocess(doc)
#inferred_vector = model.infer_vector(gensim.utils.simple_preprocess(doc),steps=125)

#Doc2Vec.similarity_unseen_docs(model,doc,doc)
#model.docvecs.similarity_unseen_docs(doc,doc)

TypeError: similarity_unseen_docs() missing 1 required positional argument: 'doc_words2'